In [2]:
import pandas as pd
from datas import *

year = 2019
list_assets, df_assets = get_zz500_stocks(f'{year}-01-01')
dfs= get_all_date_data(f'{year}-01-01', f'{year+1}-01-01', list_assets)


df_all = dfs[['date', 'asset', "close"]]
df_all['date'] = pd.to_datetime(df_all['date'])
# print(df_all)

close = df_all.pivot(index='date', columns='asset', values='close')
# close.index = pd.to_datetime(close.index)
print(close)

login success!
login respond error_code:0
login respond  error_msg:success
query_zz500 error_code:0
query_zz500  error_msg:success
logout success!
500
asset       000006  000008  000009  000012  000021  000025  000027  000028  \
date                                                                         
2019-01-02    5.15    3.88    4.30    3.98    5.68   26.64    5.15   40.25   
2019-01-03    5.15    3.84    4.24    4.01    5.64   27.00    5.15   40.33   
2019-01-04    5.26    3.91    4.37    4.10    5.78   27.29    5.22   40.26   
2019-01-07    5.34    4.07    4.45    4.13    6.03   27.39    5.23   40.18   
2019-01-08    5.32    4.14    4.52    4.13    6.03   27.77    5.27   39.85   
...            ...     ...     ...     ...     ...     ...     ...     ...   
2019-12-25    5.24    3.62    5.32    4.71   11.92   20.73    6.13   45.45   
2019-12-26    5.29    3.63    5.85    4.82   12.14   20.85    6.13   44.96   
2019-12-27    5.27    3.63    6.36    4.77   12.53   20.83    6.12   

<ipython-input-2-85ecb34d535b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['date'] = pd.to_datetime(df_all['date'])


In [3]:
###########################简单现计算的因子#######################################
# alpha = dfs[['date', 'asset', "pctChg"]]
# alpha = alpha.rename(columns={
#         "pctChg": "factor"})
# alpha['date'] = pd.to_datetime(alpha['date'])
# alpha = alpha.set_index(['date', 'asset'], drop=True)
# alpha.sort_index(inplace=True)
# print(alpha)

############################已计算好存在文件中的因子######################################
alpha_num = 5
alpha_name = 'Alphas191'

# 读取已经计算好的因子
alpha = pd.read_csv('alphas/{}/{}/alpha{:03d}.csv'.format(alpha_name, year, alpha_num))

# 筛选出今年的数据，需与股票收盘日期区间一致
alpha = alpha[(alpha['date'] >= f'{year}-01-01') & (alpha['date'] <= f'{year+1}-01-01')]

# 因子矩阵转换为一维数据(alphalens需要的格式)
alpha = alpha.melt(id_vars=['date'], var_name='asset', value_name='factor' )

# date列转为日期格式
alpha['date'] = pd.to_datetime(alpha['date'])
alpha = alpha[['date', 'asset', 'factor']]

# 设置二级索引
alpha = alpha.set_index(['date', 'asset'], drop=True)
alpha.sort_index(inplace=True)
print(alpha)

#############################因子分析#######################################

from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

ret = get_clean_factor_and_forward_returns(alpha, close,quantiles=10)
create_full_tear_sheet(ret, long_short=True)

FileNotFoundError: [Errno 2] No such file or directory: './alphas/2019/alpha005.csv'